In [1]:
import pandas as pd
import spacy
import os
import spacy
import mlflow
import mlflow.spacy
from pathlib import Path
import json
from spacy.util import minibatch, compounding
from spacy.cli.train import train
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL"))
mlflow.set_experiment("jupyter_NER")

<Experiment: artifact_location='mlflow-artifacts:/466776196716148841', creation_time=1741610247334, experiment_id='466776196716148841', last_update_time=1741610247334, lifecycle_stage='active', name='jupyter_NER', tags={}>

In [4]:
model_uri = "models:/recipe_NER@prod"
ner = mlflow.spacy.load_model(model_uri=model_uri)

/home/naabzer/develop/recipe_tm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

In [6]:
nlg_ds = pd.read_csv('../data/recipenlg/RecipeNLG_dataset.csv', converters={'ingredients': pd.eval}, nrows=10)

In [7]:
nlg_ds['ingredients'][0]

['1 c. firmly packed brown sugar',
 '1/2 c. evaporated milk',
 '1/2 tsp. vanilla',
 '1/2 c. broken nuts (pecans)',
 '2 Tbsp. butter or margarine',
 '3 1/2 c. bite size shredded rice biscuits']

In [8]:
out = [ner(line) for line in nlg_ds['ingredients'][5]]

In [11]:
def process_ingredient(ingredient_doc: spacy.tokens.doc.Doc, include_variety: bool = False):
    food_lemmas = []
    prep_lemmas = [[]]
    optional = False
    if include_variety:
        var_lemmas = [[]]
    for t in ingredient_line.ents:
        if t.label_ == 'Food':
            lemmas = [w.lemma_.lower() for w in nlp(str(t))]
            food_lemmas.append(lemmas)
            if len(prep_lemmas[-1]) > 0:
                prep_lemmas.append([])  # Add a new list of preparation that corresponds to a new food item on the same line
            if include_variety and len(var_lemmas[-1]) > 0:
                var_lemmas.append([]) 
        elif t.label_ == 'Preparation':
            lemmas = [w.lemma_.lower() for w in nlp(str(t))]
            prep_lemmas[-1] = prep_lemmas[-1] + lemmas
        elif include_variety and t.label_ == 'Variety':
            lemmas = [w.lemma_.lower() for w in nlp(str(t))]
            var_lemmas[-1] = var_lemmas[-1] + lemmas
        elif t.label_ == "Optional":
            optional = True
    ret_obj =  {'food': food_lemmas,
                'prep': prep_lemmas,
                'optional': optional}
    if include_variety:
        ret_obj['var': var_lemmas]
    return ret_obj

In [12]:
print('\n'.join([str(o) for o in out]))
for ingredient_line in out:
    process_obj = process_ingredient(ingredient_line)
    food_lemmas = process_obj['food']
    prep_lemmas = process_obj['prep']
    print('----')
    print(food_lemmas)
    print(prep_lemmas)


6 baking potatoes
1 lb. of extra lean ground beef
2/3 c. butter or margarine
6 c. milk
3/4 tsp. salt
1/2 tsp. pepper
1 1/2 c (6 oz.) shredded Cheddar cheese, divided
12 sliced bacon, cooked, crumbled and divided
4 green onion, chopped and divided
1 (8 oz.) carton sour cream (optional)
----
[['bake', 'potato']]
[[]]
----
[['beef']]
[['ground'], []]
----
[['butter']]
[[]]
----
[['milk']]
[[]]
----
[['salt']]
[[]]
----
[['pepper']]
[[]]
----
[['cheddar', 'cheese']]
[['shred'], []]
----
[['bacon']]
[['slice'], ['cook', 'crumble']]
----
[['green', 'onion']]
[['chop']]
----
[['sour', 'cream']]
[[]]
